## Agile Hardware Design
***
# Inheritance

<img src="./images/chisel_logo.svg" alt="agile hardware design logo" style="float:right"/>

Peter Hanping Chen based on
1. UC Berkley, Bootcamp, load-ivy.sc
https://github.com/freechipsproject/chisel-bootcamp/tree/master/source
2. Prof. Scott Beamer, sbeamer@ucsc.edu
[CSE 228A](https://classes.soe.ucsc.edu/cse228a/Winter24/)

## Plan for Today

* Inheritance in Scala
* Inheritance with Chisel
* Type parameterization
* Project Advice

## Loading The Chisel Library Into a Notebook

In [1]:
//interp.load.module(os.Path(s"${System.getProperty("user.dir")}/../resource/chisel_deps.sc"))
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
//val path = System.getProperty("user.dir") + "/source/chisel_deps.sc"
println("path: "+path)
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

path: /home/peter/AIU/AIU_CS800_Chisel/500_UCSC_HWD/014_Inherit/001_Code/source/load-ivy.sc


path: String = "/home/peter/AIU/AIU_CS800_Chisel/500_UCSC_HWD/014_Inherit/001_Code/source/load-ivy.sc"

In [2]:
import chisel3._
import chisel3.util._
import chiseltest._
import chiseltest.RawTester.test

import chisel3._

import chisel3.util._

import chiseltest._

import chiseltest.RawTester.test

## Why Use (Object-Oriented) Inheritance?

* Increase productivity through _reuse_
  * Less effort to reuse a component than implementing from scratch

* _Inheritance_ is most helpful for reuse between similar components
  * What about different components that behave the same in same cases?
  * What about different components that present the same interface?

* Due to diversity of potential reuse cases, Scala has a diversity of inheritance mechanisms

## Scala `class` Inheritance

* Simply use the `extend` keyword
  * Can only extend one class at a time

* Can overide fields with `override`

In [3]:
class Parent(name: String) {
    val phrase = "hello"
    
    def greet() { println(s"$phrase $name") }
}

val p = new Parent("Kate")
p.greet

class Child(name: String) extends Parent(name) {
    override val phrase = "hola"
}

val c = new Child("Pablo")
c.greet

hello Kate
hola Pablo


defined class Parent
p: Parent = ammonite.$sess.cmd2$Helper$Parent@74bf73de
defined class Child
c: Child = ammonite.$sess.cmd2$Helper$Child@2fddb34c

## Scala `abstract class` Inheritance

* Sometimes don't want to provide implementations of the inherited things
* Can't instantiate an abstract class, must inherit from it
* For multiple inheritance, will need to consider _trait_

In [4]:
abstract class Parent(name: String) {
    val phrase: String
    
    def greet() { println(s"$phrase $name") }
}

class InEnglish(name: String) extends Parent(name) {
    val phrase = "hello"
}

val e = new InEnglish("Kate")
e.greet

class InSpanish(name: String) extends Parent(name) {
    val phrase = "hola"
}

val s = new InSpanish("Pablo")
s.greet

hello Kate
hola Pablo


defined class Parent
defined class InEnglish
e: InEnglish = ammonite.$sess.cmd3$Helper$InEnglish@439eb9c8
defined class InSpanish
s: InSpanish = ammonite.$sess.cmd3$Helper$InSpanish@3546cfb6

## Example Type Hierarchy: Scala's Immutable Collections

<img src="images/collections-immutable-diagram.svg" alt="Scala's immutable collections type hierarchy" style="width:50%;align:left" />

* source:
https://docs.scala-lang.org/overviews/collections-2.13/overview.html)

## Types of Reuse in Using Chisel

* To enable agile development, we have been going after reuse in different places:

* _**Parameterized hardware generators**_ - hopefully sufficiently flexible to be used in more places
  * Both at module level as well as functionality not even wrapped in a Module

* _**Composable/customizable Bundles**_ - can reduce effort defining interfaces

* _**via Inheritance**_ - (today) similar modules can share functionality
  * Design decision of using different classes (inheritance) or more generator parameters
  * _Note:_ Chisel itself is implemented using inheritance (e.g. `extend Module`)


## Chisel Simple `abstract class` Inheritance

In [5]:
abstract class UnaryOperatorModule(width: Int) extends Module {
    def op(x: UInt): UInt
    val io = IO(new Bundle {
        val in = Input(UInt(width.W))
        val out = Output(UInt(width.W))
    })
    io.out := op(io.in)
}

class PassThruMod(width: Int) extends UnaryOperatorModule(width) {
    def op(x: UInt) = x
}

class NegMod(width: Int) extends UnaryOperatorModule(width) {
    def op(x: UInt) = ~x
}

defined class UnaryOperatorModule
defined class PassThruMod
defined class NegMod

In [6]:
//printVerilog(new PassThruMod(8))
println (getVerilog(new PassThruMod(8)))

Elaborating design...
Done elaborating.
module PassThruMod(
  input        clock,
  input        reset,
  input  [7:0] io_in,
  output [7:0] io_out
);
  assign io_out = io_in; // @[cmd4.sc 7:12]
endmodule



## Chisel Example - Building Operator Library (1/3)

In [7]:
//  c = op(a,b)
abstract class DecoupledOperator(width: Int) extends Module {
    val io = IO(new Bundle {
        val a = Flipped(Decoupled(UInt(width.W)))
        val b = Flipped(Decoupled(UInt(width.W)))
        val c = Decoupled(UInt(width.W))
    })

    def op(a: UInt, b: UInt): UInt

    val buffer = Reg(UInt(width.W))
    val full = RegInit(false.B)
    io.a.ready := !full
    io.b.ready := !full
    io.c.valid := full
    io.c.bits := buffer
    when (io.a.fire && io.b.fire && !full) {
        buffer := op(io.a.bits, io.b.bits)
        full := true.B
    }
    when (io.c.fire) {
        full := false.B
    }
}

defined class DecoupledOperator

## Chisel Example - Building Operator Library (2/3)

In [8]:
class DecoupledAdd(width: Int) extends DecoupledOperator(width) {
    def op(a: UInt, b: UInt): UInt = a + b
}

class DecoupledSub(width: Int) extends DecoupledOperator(width) {
    def op(a: UInt, b: UInt): UInt = a - b
}

object DecoupledFactory {
    def apply(op: String, width: Int): DecoupledOperator = op match {
        case "+" => new DecoupledAdd(width)
        case "-" => new DecoupledSub(width)
        case _ => throw new Exception(s"Couldn't find $op")
    }
}

defined class DecoupledAdd
defined class DecoupledSub
defined object DecoupledFactory

## Chisel Example - Building Operator Library (3/3)

In [8]:
/* printVerilog(DecoupledFactory("+", 8))

// test(DecoupledFactory("+", 8)) { c =>
//     for (cycle <- 0 until 5) {
//         c.io.a.bits.poke((cycle+1).U)
//         c.io.a.valid.poke(true.B)
//         c.io.b.bits.poke(cycle.U)
//         c.io.b.valid.poke(true.B)
//         c.io.c.ready.poke(true.B)
//         println(s"$cycle:$cycle ${c.io.c.bits.peek} ${c.io.c.valid.peek}")
//         c.clock.step()
//     }
// }
*/

In [9]:
println (getVerilog(DecoupledFactory("+", 8)))

Elaborating design...
Done elaborating.
module DecoupledAdd(
  input        clock,
  input        reset,
  output       io_a_ready,
  input        io_a_valid,
  input  [7:0] io_a_bits,
  output       io_b_ready,
  input        io_b_valid,
  input  [7:0] io_b_bits,
  input        io_c_ready,
  output       io_c_valid,
  output [7:0] io_c_bits
);
`ifdef RANDOMIZE_REG_INIT
  reg [31:0] _RAND_0;
  reg [31:0] _RAND_1;
`endif // RANDOMIZE_REG_INIT
  reg [7:0] buffer; // @[cmd6.sc 10:21]
  reg  full; // @[cmd6.sc 11:23]
  wire  _T = ~full; // @[cmd6.sc 12:19]
  wire  _T_2 = io_a_ready & io_a_valid; // @[Decoupled.scala 40:37]
  wire  _T_3 = io_b_ready & io_b_valid; // @[Decoupled.scala 40:37]
  wire [7:0] _T_8 = io_a_bits + io_b_bits; // @[cmd7.sc 2:40]
  wire  _GEN_1 = _T_2 & _T_3 & _T | full; // @[cmd6.sc 16:44 cmd6.sc 18:14 cmd6.sc 11:23]
  wire  _T_9 = io_c_ready & io_c_valid; // @[Decoupled.scala 40:37]
  assign io_a_ready = ~full; // @[cmd6.sc 12:19]
  assign io_b_ready = ~full; // @[cm

## Scala `trait`

* More flexible than `abstract class` in most ways
  * Can inherit from multiple traits
  * Can't take constructor parameters
* Sometimes refered to as _mixin_
  * Good conceptual model: think of inheriting from trait to "mix in" some needed functionality (or interface)
* Great in Chisel for adding a little functionality to different types of modules

## Example Chisel Use of `trait`

* Want to add standardized way of querying module

In [9]:
/*
// trait PrintInSim {
trait PrintInSim extends Module {
    val printEnable = IO(Input(Bool()))
    
    def msg: String

    when (printEnable) {
        printf(p"$msg\n")
    }
}

//class CounterMod extends Module with PrintInSim {
abstract class CounterMod extends Module with PrintInSim {
    val out = IO(Output(UInt(8.W)))
    def msg = "hello from counter"
    val count = Counter(255)
    out := count.value
}

class CountMed1 extends CounterMod {}

//test(new CounterMod) { c =>
test(new CounterMod1) { c =>
    c.printEnable.poke(false.B)
    c.clock.step(2)
    c.printEnable.poke(true.B)
    c.clock.step(2)
}
*/

In [9]:
/*
trait PrintInSim {
    val printEnable = IO(Input(Bool()))
    
    def msg: String

    when (printEnable) {
        printf(p"$msg\n")
    }
}

class CounterMod extends Module with PrintInSim {
    val out = IO(Output(UInt(8.W)))
    def msg = "hello from counter"
    val count = Counter(255)
    out := count.value
}

test(new CounterMod) { c =>
    c.printEnable.poke(false.B)
    c.clock.step(2)
    c.printEnable.poke(true.B)
    c.clock.step(2)
}
*/

## Scala Class Mechanism Recap (& Common Uses in Chisel)

* **`class`** - "regular" class, most commonly used
    * With Chisel, use for most things including modules, bundles, etc...
* **`object`** - singleton object, can be companion object
    * Can get multiple constructors (via factory method)
    * Can also group stateless code blocks or constants
* **`case class`** - restricted form of class with some functionality built-in
    * With Chisel, great for parameters and for use with pattern matching
* **`abstract class`** - virtual class useful when inherited
    * With Chisel, enables sharing functionality across different classes
* **`trait`** - like an interface in other languages, allows multiple inheritance
    * With Chisel, useful for "mixing-in" functionality
    * Not as rigid as inheriting from an abstract class

## Templating Types

* Possible to parameterize a type (e.g. a generic) in Scala
* Typically want to use _type bounds_ to ensure functionality is there
* Can sometimes come run into issue of [_type erasure_](https://squidarth.com/scala/types/2019/01/11/type-erasure-scala.html)

## Chisel with Templated Type

* With Chisel, usually want to specify _type bounds_ to ensure is hardware
* Need to explicitly pass _gen_ to get type constructor

In [10]:
class GenericPassThru[T <: chisel3.Data](gen: T) extends Module {
    val io = IO(new Bundle {
        val in = Input(gen)
        val out = Output(gen)
    })
    io.out := io.in
}

defined class GenericPassThru

In [11]:
//printVerilog(new GenericPassThru(UInt(8.W)))
println (getVerilog (new GenericPassThru(UInt(8.W))))

Elaborating design...
Done elaborating.
module GenericPassThru(
  input        clock,
  input        reset,
  input  [7:0] io_in,
  output [7:0] io_out
);
  assign io_out = io_in; // @[cmd9.sc 6:12]
endmodule



## Chisel Type Hierarchy

<img src="images/chisel_hierarchy.svg" alt="Chisel's type hierarchy" style="width:50%;align:left" />

- source:
https://github.com/chipsalliance/chisel3)

## Templating Our Queue (from last lecture)

In [12]:
class MyQueueV7[T <: chisel3.Data](numEntries: Int, gen: T, pipe: Boolean=true) extends Module {
    val io = IO(new Bundle {
        val enq = Flipped(Decoupled(gen))
        val deq = Decoupled(gen)
    })
    require(numEntries > 1)
//     require(isPow2(numEntries))
    val entries = Mem(numEntries, gen)
    val enqIndex = Counter(numEntries)
    val deqIndex = Counter(numEntries)
    val maybeFull = RegInit(false.B)
    val indicesEqual = enqIndex.value === deqIndex.value
    val empty = indicesEqual && !maybeFull
    val full = indicesEqual && maybeFull
    if (pipe)
        io.enq.ready := !full || io.deq.ready
    else
        io.enq.ready := !full
    io.deq.valid := !empty
    io.deq.bits := entries(deqIndex.value)
    when (io.deq.fire =/= io.enq.fire) {
        maybeFull := io.enq.fire
    }
    when (io.deq.fire) {
        deqIndex.inc()
    }
    when (io.enq.fire) {
        entries(enqIndex.value) := io.enq.bits
        enqIndex.inc()
    }
}

defined class MyQueueV7

In [12]:
//println (getVerilog (new MyQueueV7(1, 1.T, true)))

In [13]:
class QueueModel(numEntries: Int, pipe: Boolean=true) {
    val mq = scala.collection.mutable.Queue[Int]()

    var deqReady = false  // set externally
    def deqValid() = mq.nonEmpty
    // be sure to call attemptDeq before attemptEnq within a cycle
    def attemptDeq() = if (deqReady && deqValid) Some(mq.dequeue()) else None
    
    def enqReady() = mq.size < numEntries-1 || 
                    (mq.size == numEntries-1 && !deqReady) ||
                    (mq.size == numEntries-1 && deqReady && pipe)
    def attemptEnq(elem: Int): Unit = if (enqReady()) mq += elem    // implies enqValid
}

defined class QueueModel

In [14]:
def simCycle(qm: QueueModel, c: MyQueueV7[UInt], enqValid: Boolean, deqReady: Boolean, enqData: Int=0) {
    qm.deqReady = deqReady
    c.io.deq.ready.poke(qm.deqReady.B)
    c.io.deq.valid.expect(qm.deqValid.B)
    val deqResult = qm.attemptDeq()
    if (deqResult.isDefined)
        c.io.deq.bits.expect(deqResult.get.U)
    c.io.enq.ready.expect(qm.enqReady.B)
    c.io.enq.valid.poke(enqValid.B)
    c.io.enq.bits.poke(enqData.U)
    if (enqValid)
        qm.attemptEnq(enqData)
    c.clock.step()
    println(qm.mq)
}

test(new MyQueueV7(3, UInt(8.W))) { c =>
    val qm = new QueueModel(3)
    simCycle(qm, c, false, false)
    simCycle(qm, c, true, false, 1)
    simCycle(qm, c, true, false, 2)
    simCycle(qm, c, true, false, 3)
    simCycle(qm, c, false, true)
}

Elaborating design...
Done elaborating.
Queue()
Queue(1)
Queue(1, 2)
Queue(1, 2, 3)
Queue(2, 3)
test MyQueueV7 Success: 0 tests passed in 7 cycles in 0.072530 seconds 96.51 Hz


defined function simCycle

## Project Suggestions

### GOAL: gain experience developing/revising a generator

### Things to Consider
* App or domains you are interested in (to speed up getting started)
* Opportunities for parameterization / generation (avoid fixed single instance)
* Allows for incremental progress (avoid all or nothing)
* Prior work on the topic? In Chisel? How can yours be different?
* How will you get started? What do defer? Code to bootstrap?

## Past Project Highlights

* Cipher / hashes (e.g. 3GPP LTE)
* Neural net (generator)
* TPU (systolic matrix multiplication)
* Bloom filter (generator)
* Mandlebrot (fractal)
* Smith-Waterman (sequence alignment)
* Image histogram equalization